In [1]:
from hmr4d import os_chdir_to_proj_root

os_chdir_to_proj_root()

In [4]:
VERSION = "v2"


import torch
import numpy as np
from hmr4d.utils.wis3d_utils import make_wis3d, add_motion_as_lines
from pathlib import Path


output_dir = Path(f"outputs/figure/{VERSION}")
output_dir.mkdir(parents=True, exist_ok=True)

data = torch.load("outputs/figure/pipeline_j3d.pth", weights_only=False)

if False:
    j3d = data["j3ds"]
    wis3d = make_wis3d(output_dir="outputs/wis3d_4D", name="high_res_pick_frame")
    add_motion_as_lines(j3d, wis3d)
    if True:
        from_dir = Path("outputs/wis3d_4D/high_res_pick_frame")
        to_dir = Path("outputs/wis3d_4D/high_res_pick_frame_joints22")
        to_dir.mkdir(parents=True, exist_ok=True)
        # Move from_dir/{frame_id:05d}/meshes/joints22.ply to to_dir/{frame_id:05d}_joints22.ply
        for frame_id in range(j3d.shape[0]):
            from_path = from_dir / f"{frame_id:05d}" / "meshes" / "joints22.ply"
            to_path = to_dir / f"{frame_id:05d}_joints22.ply"
            from_path.rename(to_path)

if VERSION == "v1":
    frame_ids = [3, 36]
elif VERSION == "v2":
    frame_ids = [3, 20, 36]

j3d_selected = data["j3ds"][frame_ids][:, :22]
f2d_selected = data["c_motion_in3d"][frame_ids][:, :22]
T_w2c_selected = data["T_w2c"][frame_ids]

In [9]:
from hmr4d.utils.wis3d_utils import (
    make_wis3d,
    create_skeleton_mesh,
    KINEMATIC_CHAINS,
    get_const_colors,
    color_schemes,
    add_motion_as_lines,
)
import numpy as np
import trimesh


def create_colored_balls(j3d, radius=1.0, subdivisions=3, color_rgb=None):
    """Create J balls, the center of each ball is the joint position.
    j3d: (J, 3)
    """
    j3d = j3d.detach().cpu().numpy()
    assert color_rgb is not None
    color = color_rgb  # (3,)

    # Create spheres for each joint
    spheres = []
    for joint in j3d:
        sphere = trimesh.creation.icosphere(subdivisions=subdivisions, radius=radius)
        sphere.apply_translation(joint)
        spheres.append(sphere)

    combined = trimesh.util.concatenate(spheres)
    vertices = combined.vertices
    faces = combined.faces
    vertex_colors = np.tile(color, (vertices.shape[0], 1))
    combined.visual.vertex_colors = vertex_colors

    return combined


def convert_j3d_as_mesh(motion, skeleton_type="smpl22", color_rgb=None):
    assert color_rgb is not None
    motion = motion.detach().cpu()
    kinematic_chain = KINEMATIC_CHAINS[skeleton_type]
    color_names = ["red", "green", "blue", "cyan", "magenta"]
    s_points = []
    e_points = []
    m_colors = []
    length = motion.shape[0]
    assert length == 1
    for chain in kinematic_chain:
        s_points.append(motion[:, chain[:-1]])
        e_points.append(motion[:, chain[1:]])

    s_points = torch.cat(s_points, dim=1)  # (L, ?, 3)
    e_points = torch.cat(e_points, dim=1)
    # m_colors = torch.cat(m_colors, dim=1)
    m_colors = torch.ones_like(s_points) * torch.Tensor(color_rgb)

    vertices, faces, vertex_colors = create_skeleton_mesh(
        s_points[0], e_points[0], radius=0.02, color=m_colors[0], resolution=21
    )
    vertex_colors = vertex_colors.cpu().numpy().astype(np.uint8)

    mesh_line = trimesh.Trimesh(vertices, faces, vertex_colors=vertex_colors)
    return mesh_line


wis3d = make_wis3d(output_dir="outputs/wis3d_4D", name=f"high_res_{VERSION}")

# Draw frames
if VERSION == "v1":
    rgb_colors = [[180, 200, 253], [0, 81, 255]]
elif VERSION == "v2":
    rgb_colors = [[181, 204, 255], [92, 143, 255], [0, 81, 255]]

for idx in range(len(frame_ids)):
    color_rgb = [n for n in rgb_colors[idx]]
    mesh_line = convert_j3d_as_mesh(j3d_selected[idx][None], skeleton_type="smpl22", color_rgb=color_rgb)
    mesh_ball = create_colored_balls(j3d_selected[idx], radius=0.035, subdivisions=3, color_rgb=color_rgb)
    wis3d.add_mesh(mesh_line, name=f"frame{idx}_skeleton")
    wis3d.add_mesh(mesh_ball, name=f"frame{idx}_ball")

if False:
    for v in range(4):
        add_motion_as_lines(f2d_frame1[[v]], wis3d, name=f"high_res_2d_{v}")

In [10]:
# Make camera closer to the subject
from hmr4d.utils.wis3d_utils import draw_T_w2c


def get_a_closer_T_w2c(T_w2c_this_frame, j3d_root, ratio=0.20):
    T_w2c_this_frame = T_w2c_this_frame.clone()
    cam_center = (T_w2c_this_frame[:, :3, :3].transpose(1, 2) @ -T_w2c_this_frame[:, :3, [3]]).squeeze(-1)
    new_cam_center = j3d_root + ratio * (cam_center - j3d_root)
    new_T_w2c_this_frame = T_w2c_this_frame.clone()  # (V, 4, 4)
    new_T_w2c_this_frame[:, :3, 3] = -torch.einsum("bij,bj->bi", new_T_w2c_this_frame[:, :3, :3], new_cam_center)
    return new_T_w2c_this_frame


for idx in range(len(frame_ids)):
    T_w2c_frame_closer = get_a_closer_T_w2c(T_w2c_selected[idx], j3d_selected[idx][0])
    # draw_T_w2c(wis3d, T_w2c_selected[idx], name=f"T_w2c_frame{idx}", all_in_one=True)
    draw_T_w2c(wis3d, T_w2c_frame_closer, name=f"T_w2c_frame{idx}_closer", all_in_one=True)
    np.save(output_dir / f"new_T_w2c_frame{idx}.npy", T_w2c_frame_closer.numpy())

### Draw 2D skeleton Image

In [11]:
import matplotlib.pyplot as plt
from pathlib import Path
from hmr4d.utils.geo_transform import project_p2d, apply_T_on_points


def draw_2d_skeleton(j3d_frame1_cam, save_path=None, rgb_color=None):
    if rgb_color is None:
        rgb_color = [i / 255 for i in color_choices["blue"]]
    size = 400
    canvas = np.ones((size, size, 3))
    f = 800
    K = torch.tensor([[f, 0, size / 2], [0, f, size / 2], [0, 0, 1]])

    f2d_frame1_cam = project_p2d(j3d_frame1_cam, K, size)
    f2d_frame1_cam = f2d_frame1_cam.detach().cpu().numpy()[:22, :]

    # Plot lines
    kinematic_chain = KINEMATIC_CHAINS["smpl22"]
    for chain in kinematic_chain:
        for i in range(len(chain) - 1):
            x1, y1 = f2d_frame1_cam[chain[i]]
            x2, y2 = f2d_frame1_cam[chain[i + 1]]
            plt.plot([x1, x2], [y1, y2], "-", color=rgb_color, linewidth=2)

    # plot circle
    for i in range(f2d_frame1_cam.shape[0]):
        x, y = f2d_frame1_cam[i]
        plt.plot(x, y, "o", markerfacecolor=[1, 1, 1], markeredgecolor=rgb_color)  # 空心圆，边缘有颜色

    plt.imshow(canvas)
    plt.gcf().set_dpi(300)  # 设置dpi高一些
    plt.axis("off")  # 关闭坐标轴
    if save_path is not None:
        plt.savefig(save_path, format="png", bbox_inches="tight", pad_inches=0, dpi=300)  # 保存图像为jpg
    plt.clf()
    # plt.show()


# Frame
for idx in range(len(frame_ids)):
    for i in range(4):
        rgb_color = [n / 255 for n in rgb_colors[idx]]
        j3d_frame1_cam = apply_T_on_points(j3d_selected[idx], T_w2c_selected[idx][i])
        save_path = output_dir / f"2d_skeleton_frame{idx}_{i}.png"
        Path(save_path).parent.mkdir(parents=True, exist_ok=True)
        draw_2d_skeleton(j3d_frame1_cam, save_path, rgb_color)

<Figure size 1920x1440 with 0 Axes>